In [ ]:
import json
#read baseline_results.json
with open('testset/question_generation_results.jsonl') as f:
    predictions = json.load(f)
    print(predictions)

In [ ]:
import pandas as pd
# Convert data into a DataFrame

def createDF(predictions,filename):
        
    # Convert data into DataFrame
    tasks = ['Timeline Reorder', 'Multiple Information Retrieval', 'Comprehension and Reasoning', 'Computation']
    correct_matched = [predictions['true'][task.lower().replace(' ', '_')]['matched'] for task in tasks]
    correct_total = [predictions['true'][task.lower().replace(' ', '_')]['total'] for task in tasks]
    incorrect_matched = [predictions['false'][task.lower().replace(' ', '_')]['matched'] for task in tasks]
    incorrect_total = [predictions['false'][task.lower().replace(' ', '_')]['total'] for task in tasks]

    df = pd.DataFrame({
        'Task': tasks,
        'Correct Matched': correct_matched,
        'Correct Total': correct_total,
        'Incorrect Matched': incorrect_matched,
        'Incorrect Total': incorrect_total
    })

    
    # Calculate accuracy and percentage of chunk matches
    df['Total Count'] = df['Correct Total'] + df['Incorrect Total']
    df['Accuracy'] = df['Correct Total'] / df['Total Count']
    df['Correct Prediction Chunk Match'] = df['Correct Matched'] / df['Correct Total']
    df['InCorrect Prediction Chunk Match'] = df['Incorrect Matched'] / df['Incorrect Total']

    # Format as percentages
    df['Accuracy'] = (df['Accuracy'] * 100).round(2).astype(str) + '%'
    df['Correct Prediction Chunk Match'] = (df['Correct Prediction Chunk Match'] * 100).round(2).astype(str) + '%'
    df['InCorrect Prediction Chunk Match'] = (df['InCorrect Prediction Chunk Match'] * 100).round(2).astype(str) + '%'
    df1=df[['Task', 'Accuracy', 'Correct Prediction Chunk Match', 'InCorrect Prediction Chunk Match']].T
    df1["Method"]=filename
    df1=df1.reset_index()
    cols = df1.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    df1 = df1[cols]
    return df1


import json
#read baseline_results.json
output_files=['testset/testset_baseline_results_test.jsonl','testset/testset_questions_mistral_large_results_test.jsonl','testset/testset_question_union_results_test.jsonl','testset/summarization_icl_results_test.jsonl','testset/testset_summary_only_results_test.jsonl','testset/testset_summary_union_results_test.jsonl', 'testset/testset_union_results_test.jsonl']

df_final=pd.DataFrame()
for output_file in output_files:
    with open(output_file) as f:
        predictions = json.load(f)
        #extract fileame
        output_file_name=output_file.split('/')[1].split('.')[0]
        print(output_file_name)
        df=createDF(predictions,output_file_name)
        df_final=pd.concat([df_final,df],axis=0)


df_final.to_csv('testset/summary.csv',index=False)

In [2]:
%load_ext autoreload
%autoreload 2


from rag_components import *
from llama_index.core.retrievers import VectorIndexRetriever

import re

c:\Users\Abhishek Goyal\AppData\Local\Programs\Python\Python311\Lib\site-packages\datasets\load.py:1486: FutureWarning: The repository for bigainlco/LooGLE contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/bigainlco/LooGLE
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [4]:
# llm = Settings.llm
from datasets import load_dataset
longdep_qa_ds = load_dataset("bigainlco/LooGLE", "longdep_qa", split="test")
n_questions = sum([len(eval(env["qa_pairs"])) for env in longdep_qa_ds])
debug_lim = 206
write_file = "output/baseline_SOTA.jsonl"
example_list = []
temp_list = []
from tqdm.auto import tqdm
with tqdm(total=debug_lim, position=0, desc="Answering questions") as pbar:
    for environment in longdep_qa_ds:
        context = environment["input"]
        title = environment["title"]
        title = f"{title}_chunksize{256}"
        print(environment)
        
        qa_pairs = eval(environment["qa_pairs"])
        for question_dict in qa_pairs:
            question = question_dict["Q"]
            ground_truth = question_dict["A"]
            question_type = question_dict["type"]
            question_context = question_dict["S"]
            #combine context from question_context
            full_context = " ".join(question_context)
            prompt = generate_qa_prompt(full_context, question)
            response = mistral_large.complete(prompt).text

            generated_answer =response
            prompt = f"""\Question: {question}\n
                \Answer: {ground_truth}\n
                \Predicted Answer: {generated_answer}\n\n
                \given the question and the ground truth answer, is the predicted answer correct?\n
                \True or False?: """
            prompt = re.sub(r"\s+", " ", prompt)
            response = mistral_large.complete(prompt).text.lower()
            # Create a dictoinary to store the question, question type, question context, and the response
            output = {
                "question": question,
                "question_type": question_type,
                "question_context": question_context,
                "response": generated_answer
            }
            if "false" in response:
                output["correct"] = False
            else:
                output["correct"] = True

            # Write the output to a file
            with open(write_file, "a") as output_file:
                output_file.write(json.dumps(output) + "\n")

            pbar.update(1)
            if pbar.n >= debug_lim:
                break
        if pbar.n >= debug_lim:
            break

c:\Users\Abhishek Goyal\AppData\Local\Programs\Python\Python311\Lib\site-packages\datasets\load.py:1486: FutureWarning: The repository for bigainlco/LooGLE contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/bigainlco/LooGLE
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Answering questions:   0%|          | 0/206 [00:00<?, ?it/s]

{'input': ' Early life. Picardo was born in Jerez de la Frontera, in the Province of Cádiz in Andalucía, Spain on 18 June 1919. His father was Alvaro Picardo de Celis and his mother\'s family name was Castellón. He had four brothers, one of whom died in infancy. His father died in 1929 when Picardo was ten years old. With his mother and his brothers he moved to Madrid, Spain. He enrolled at the newly created Instituto de Bachillerato Cervantes for his high school education. On completing school he initially wanted to join the navy, but was frustrated by the closure of the military academies in Madrid during the Second Spanish Republic. He turned to the study of law, but was frustrated again, this time by the start of the Spanish Civil War in July 1936 when he was in the middle of his course. He had just celebrated his seventeenth birthday. Training in architecture. To avoid being evacuated from Madrid when the Spanish Civil War began, Picardo joined the studio of the architect Luis Moy